# Çözücüler ⚙️

Bu egzersizde, farklı `çözücülerin` `LogisticRegression` modelleri üzerindeki etkilerini araştıracaksınız.

👇 Veri kümesini içe aktarmak için aşağıdaki kodu çalıştırın

In [1]:
import pandas as pd

df = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/solvers_dataset.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol,quality rating
0,9.47,5.97,7.36,10.17,6.84,9.15,9.78,9.52,10.34,8.80,6
1,10.05,8.84,9.76,8.38,10.15,6.91,9.70,9.01,9.23,8.80,7
2,10.59,10.71,10.84,10.97,9.03,10.42,11.46,11.25,11.34,9.06,4
3,11.00,8.44,8.32,9.65,7.87,10.92,6.97,11.07,10.66,8.89,8
4,12.12,13.44,10.35,9.95,11.09,9.38,10.22,9.04,7.68,11.38,3


- Veri kümesi farklı şaraplardan oluşmaktadır 🍷
- Özellikler şarapların farklı niteliklerini tanımlar 
- Hedef 🎯 bir uzman tarafından verilen kalite değerlendirmesidir

## 1. Hedef mühendisliği

Bu bölümde, değerlendirmeleri ikili bir hedefe dönüştüreceksiniz.

👇 Her değerlendirme için kaç gözlem bulunmaktadır?

In [2]:
import pandas as pd
import numpy as np
import time 
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

In [6]:
df.shape

(100000, 11)

In [4]:
df['quality rating'].value_counts().sort_index()

quality rating
1     10090
2     10030
3      9838
4      9928
5     10124
6      9961
7      9954
8      9977
9      9955
10    10143
Name: count, dtype: int64

❓ Hedefi ikili sınıflandırma görevine dönüştürerek `y` oluşturun, burada 6'nın altındaki kalite değerlendirmeleri kötü [0], 6 ve üzeri değerlendirmeler iyi [1] olacak

In [ ]:
y = df['quality rating'].apply(lambda x: 0 if x < 6 else 1)

y.value_counts()

quality rating
0    50010
1    49990
Name: count, dtype: int64

❓ Yeni ikili hedefin sınıf dengesini kontrol edin

In [9]:
y.value_counts(normalize=True)

quality rating
0    0.5001
1    0.4999
Name: proportion, dtype: float64

❓ Özellikleri normalleştirerek `X`'inizi oluşturun. Bu farklı çözücülerin adil karşılaştırılmasına olanak sağlayacaktır.

In [10]:
X_raw = df.drop(columns=['quality rating'])
scaler = MinMaxScaler()
X = scaler.fit_transform(X_raw)

print(X.shape)

(100000, 10)


## 2. LogisticRegression çözücüleri

❓ Lojistik Regresyon modelleri farklı **çözücüler** kullanılarak optimize edilebilir. Mevcut çözücülerin karşılaştırmasını yapın:
- Uyum süresi - hangi çözücü **en hızlı**?
- Kesinlik - kesinlik puanları **ne kadar farklı**?

Lojistik Regresyon için mevcut çözücüler: `['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']`
 
Bu 5 çözücü hakkında daha fazla bilgi için [bu Stack Overflow konusuna](https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions) göz atın

In [16]:
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
results = []

best_time = float('inf')
fastest_solver = ""

for solver in solvers:
    start_time = time.time()
    
    model = LogisticRegression(solver=solver, max_iter=2000, random_state=42)
    model.fit(X, y)
    
    end_time = time.time()
    duration = end_time - start_time
    
    y_pred = model.predict(X)
    acc = accuracy_score(y, y_pred)
    
    if duration < best_time:
        best_time = duration
        fastest_solver = solver
    
    results.append({'Solver': solver, 'Accuracy': acc, 'Time': duration})
    
    print(f"{solver:<15} {acc:.4f}     {duration:.6f}")

print("-" * 35)
print(f"🏆 En Hızlı Solver: {fastest_solver}")

newton-cg       0.8613     0.286285
lbfgs           0.8612     0.116240
liblinear       0.8610     0.267713
sag             0.8613     0.590684
saga            0.8613     1.181262
-----------------------------------
🏆 En Hızlı Solver: lbfgs


In [13]:
# YOUR ANSWER
fastest_solver = "lbfgs"

<details>
    <summary>ℹ️ Yorumumuz için buraya tıklayın</summary>

Maliyet fonksiyonumuz 5 çözücünün de bulduğu global bir minimuma sahip olacak kadar "kolay" olduğundan, tüm çözücüler benzer kesinlik puanları üretmelidir. Derin Öğrenme'de olduğu gibi çok karmaşık maliyet fonksiyonları için, farklı çözücüler kayıp fonksiyonunun farklı değerlerinde durabilir.

**Şarap veri kümesi**
    
Mevcut veri kümesinde sklearn'in <a href="https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html">permutation_importance</a> ile özellik önemini kontrol ederseniz, birçok özelliğin neredeyse 0 önemine sahip olduğunu göreceksiniz. Liblinear çözücü, bir defada sadece *bir* yön boyunca hareket eder ve diğerlerini L1 düzenlileştirme ile düzenler (yani, beta değerlerini 0'a ayarlar), bu da birçok özelliğin hedefi tahmin etmede o kadar da önemli olmadığı bir veri kümesi için iyi bir uyum sağlayabilir.

❗️En iyi çözücüyü arama maliyeti vardır. Varsayılanla (`lbfgs`) devam etmek genel olarak en çok zaman tasarrufu sağlayabilir, sklearn başlamak için hangi çözücüyü seçeceğiniz konusunda fikir vermek için bu tabloyu sunar: 

<img src="https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/solvers-chart.png" width=700>

</details>

###  🧪 Kodunuzu test edin

In [14]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'solvers',
    fastest_solver=fastest_solver
)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/ataka/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/ataka/S16D4-S-data-solvers/tests
plugins: dash-3.3.0, anyio-4.8.0, typeguard-4.4.2
collecting ... collected 1 item

test_solvers.py::TestSolvers::test_fastest_solver PASSED                 [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/solvers.pickle

git commit -m 'Completed solvers step'

git push origin master



## 3. Stokastik Gradyan İnişi

Lojistik Regresyon modelleri ayrıca Stokastik Gradyan İnişi ile de optimize edilebilir.

❓ **Stokastik Gradyan İnişi** ile optimize edilmiş bir Lojistik Regresyon modelini değerlendirin. Kesinlik puanı ve eğitim süresi 2. bölümde eğitilen modellerin performansı ile nasıl karşılaştırılır?

<details>
<summary>💡 İpucu</summary>

- Takılırsanız, [SGDClassifier belgelerine](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) bakın!

</details>

In [18]:
sgd_model = SGDClassifier(loss='log_loss', penalty='l2', random_state=42)

start_time = time.time()
sgd_model.fit(X, y)
end_time = time.time()

sgd_duration = end_time - start_time
sgd_acc = sgd_model.score(X, y)

print(f"Accuracy: {sgd_acc:.4f} | Time: {sgd_duration:.6f} sn")

Accuracy: 0.8582 | Time: 0.206406 sn


☝️ SGD modeli, benzer performans için en kısa sürelerden birine sahip olmalıdır (hatta `liblinear`'dan bile daha kısa olabilir). Bu, Gradyan İnişinin her dönemini aynı anda 100k satırı belleğe yüklemek yerine tek bir satırda gerçekleştirmenin doğrudan bir etkisidir.

## 4. Tahminler

❓ En iyi modeli (kısa uyum süresi ve yüksek kesinlik ile dengelenen) kullanarak aşağıdaki şarabın ikili kalitesini (0 veya 1) tahmin edin. Şunları kaydedin:
- `predicted_class`
- `predicted_proba_of_class` (yani modeliniz 1 sınıfını tahmin ettiyse, 1'in sınıf olması gerektiğine inanma olasılığı nedir, 0 ile 1 arasında olmalıdır)

In [19]:
new_wine = pd.read_csv('https://d32aokrjazspmn.cloudfront.net/materials/solvers_new_wine.csv')
new_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol
0,9.54,13.5,12.35,8.78,14.72,9.06,9.67,10.15,11.17,12.17


In [25]:
new_wine_scaled = scaler.transform(new_wine)

final_model = LogisticRegression(solver='liblinear', random_state=42)
final_model.fit(X, y) 

predicted_class = final_model.predict(new_wine_scaled)[0] 
predicted_proba_of_class = final_model.predict_proba(new_wine_scaled)[0][0] 

print(f"Tahmin Edilen Sınıf: {predicted_class}")
print(f"1 Olma Olasılığı: {predicted_proba_of_class:.4f}")

Tahmin Edilen Sınıf: 0
1 Olma Olasılığı: 0.9670


# 🏁  Kodunuzu kontrol edin ve notebook'unuzu gönderin

In [26]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'new_data_prediction',
    predicted_class=predicted_class,
    predicted_proba_of_class=predicted_proba_of_class
)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/ataka/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/ataka/S16D4-S-data-solvers/tests
plugins: dash-3.3.0, anyio-4.8.0, typeguard-4.4.2
collecting ... collected 2 items

test_new_data_prediction.py::TestNewDataPrediction::test_predicted_class PASSED [ 50%]
test_new_data_prediction.py::TestNewDataPrediction::test_predicted_proba PASSED [100%]

============================== 2 passed in 0.24s ===============================


💯 You can commit your code:

git add tests/new_data_prediction.pickle

git commit -m 'Completed new_data_prediction step'

git push origin master

